In [1]:
from flask import Flask, request, jsonify, render_template
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
from filter import *
import os
import click

global filename
global geojson_df
global filter_code
global is_bio


In [2]:
def get_filename():
    filepath = "data/RPG_2-2__GPKG_LAMB93_FXX_2023-01-01/PARCELLES_GRAPHIQUES_4326.gpkg"
    filepath = "data/rpg-bio-2022-national/rpg-bio-2022-national.shp"
    print("Filepath:", filepath)
    return filepath

In [3]:
filename = get_filename()
geojson_df = gpd.read_file(filename)

Filepath: data/rpg-bio-2022-national/rpg-bio-2022-national.shp


KeyboardInterrupt: 

In [2]:
data1 = pd.read_pickle("data/rpg-bio-2022-national/rpg-bio-2022-national.pkl")
data2 = pd.read_pickle("data/RPG_2-2__GPKG_LAMB93_FXX_2023-01-01/PARCELLES_GRAPHIQUES_4326.pkl")
# data = pd.read_pickle("data/merged_data.pkl")

In [3]:
data1.head()

,gid,millesime,code_cultu,lbl_cultur,grp_cultur,surface_ha,code_commu,lbl_commun,code_epci,code_depar,code_regio,zone_geo,geometry
0,4420687,2022,PPH,Prairie permanente - herbe prÃ©dominante (ress...,Prairies permanentes,0.38,38513,Treffort,200030658.0,38,84,France mÃ©t.,"POLYGON ((5.67141 44.90419, 5.67144 44.90409, ..."
1,333,2022,PTR,Autre prairie temporaire de 5 ans ou moins,Prairies temporaires,2.13,01188,Illiat,200070118.0,01,84,France mÃ©t.,"POLYGON ((4.87139 46.18659, 4.87152 46.18600, ..."
2,337,2022,BTH,BlÃ© tendre dâhiver,BlÃ© tendre,6.10,01136,Cruzilles-lÃ¨s-MÃ©pillat,200070555.0,01,84,France mÃ©t.,"POLYGON ((4.87175 46.22321, 4.87171 46.22314, ..."
3,464,2022,BTH,BlÃ© tendre dâhiver,BlÃ© tendre,1.32,01285,Parcieux,200042497.0,01,84,France mÃ©t.,"POLYGON ((4.84690 45.92268, 4.84606 45.92260, ..."
4,480,2022,BTH,BlÃ© tendre dâhiver,BlÃ© tendre,2.99,01398,Savigneux,200042497.0,01,84,France mÃ©t.,"POLYGON ((4.85280 46.00945, 4.85280 46.00948, ..."


In [3]:
# print("Data1:", data1.head(3))

# print("\n Data2:", data2.head(3))
data1['type'] = 'bio'
data1['source'] = 'rpg-bio-2022-national'
data2['source'] = 'RPG_2-2__GPKG_LAMB93_FXX_2023-01-01'
data2['type'] = 'conventional'
data2.head(3)
data2.rename(columns={"CODE_CULTU": "code_cultu" ,"SURF_PARC" : 'surface_ha' , "CODE_GROUP" : "code_group" }, inplace=True)

data2.head(3)

,ID_PARCEL,surface_ha,code_cultu,code_group,CULTURE_D1,CULTURE_D2,geometry,source,type
0,1,5.01,ORH,3,,,"POLYGON ((1.88543 50.53479, 1.88958 50.53635, ...",RPG_2-2__GPKG_LAMB93_FXX_2023-01-01,conventional
1,2,0.43,TRN,6,,,"POLYGON ((0.71374 44.50301, 0.71374 44.50301, ...",RPG_2-2__GPKG_LAMB93_FXX_2023-01-01,conventional
2,3,2.34,BTH,1,,,"POLYGON ((2.35549 49.84055, 2.35472 49.84088, ...",RPG_2-2__GPKG_LAMB93_FXX_2023-01-01,conventional


In [4]:
data1 = data1[['code_cultu', 'surface_ha', 'geometry', 'type', 'source']]
data2 = data2[['code_cultu', 'surface_ha', 'geometry', 'type', 'source']]

data = pd.concat([data1, data2], ignore_index=True)
data.head(3)

# save to pkl 
# data.to_pickle("data/merged_data.pkl")

,code_cultu,surface_ha,geometry,type,source
0,PPH,0.38,"POLYGON ((5.67141 44.90419, 5.67144 44.90409, ...",bio,rpg-bio-2022-national
1,PTR,2.13,"POLYGON ((4.87139 46.18659, 4.87152 46.18600, ...",bio,rpg-bio-2022-national
2,BTH,6.10,"POLYGON ((4.87175 46.22321, 4.87171 46.22314, ...",bio,rpg-bio-2022-national


In [5]:
data.to_pickle("data/merged_data.pkl")

NameError: name 'data' is not defined

In [6]:
data_import = pd.read_pickle("data/merged_data.pkl")

In [16]:
data_import.tail(3)
new_data = pd.concat([data_import.head(10), data_import.tail(10)], ignore_index=True)

In [18]:
new_data.to_pickle("data/test.pkl")
new_data = pd.read_pickle("data/test.pkl")
new_data


,code_cultu,surface_ha,geometry,type,source
0,PPH,0.38,"POLYGON ((5.67141 44.90419, 5.67144 44.90409, ...",bio,rpg-bio-2022-national
1,PTR,2.13,"POLYGON ((4.87139 46.18659, 4.87152 46.18600, ...",bio,rpg-bio-2022-national
2,BTH,6.10,"POLYGON ((4.87175 46.22321, 4.87171 46.22314, ...",bio,rpg-bio-2022-national
3,BTH,1.32,"POLYGON ((4.84690 45.92268, 4.84606 45.92260, ...",bio,rpg-bio-2022-national
4,BTH,2.99,"POLYGON ((4.85280 46.00945, 4.85280 46.00948, ...",bio,rpg-bio-2022-national
5,BOP,0.34,"POLYGON ((5.84068 45.39709, 5.84065 45.39734, ...",bio,rpg-bio-2022-national
6,BTP,1.30,"POLYGON ((4.85359 45.92239, 4.85360 45.92243, ...",bio,rpg-bio-2022-national
7,PTR,5.66,"POLYGON ((4.85436 45.97779, 4.85397 45.97975, ...",bio,rpg-bio-2022-national
8,MCR,3.66,"POLYGON ((4.86287 46.01325, 4.86285 46.01284, ...",bio,rpg-bio-2022-national
9,MIE,1.43,"POLYGON ((4.86231 46.15366, 4.86238 46.15364, ...",bio,rpg-bio-2022-national


In [7]:
data = geojson_df.to_crs(epsg=4326)
filter_code = "code_cultu" if "code_cultu" in geojson_df.columns else "CODE_CULTU"
is_bio = "bio" if "code_cultu" in geojson_df.columns else "non_bio"
# if filter_code == "CODE_CULTU":
#     geojson_df = geojson_df.to_crs(epsg=4326)


In [8]:

def get_data_category():
    # Read the JSON file and send its content as the response
    json_filename = f"{os.path.splitext(filename)[0]}.json"
    # Extract unique values from the specified column
    unique_values = geojson_df[filter_code].unique()
    # Create a dictionary with keys as 1, 2, 3, ... and values as the unique values
    values_dict = {str(i+1): value for i, value in enumerate(unique_values)}
    # Save to JSON file
    with open(json_filename, 'w') as f:
        json.dump(values_dict, f)

    with open(json_filename, "r") as f:
        code_cultu_data = json.load(f)
        print("filname", json_filename)

    response = {"data_file_name": filename, "code_cultu_data": code_cultu_data}
    return response


def filter_geojson(code_cultu, surface_ha_min=None, surface_ha_max=None):
    filter_surface = "surface_ha" if "surface_ha" in geojson_df.columns else "SURF_PARC"
    filtered_df = geojson_df[
        (geojson_df[filter_code] == code_cultu)
        & ((surface_ha_min is None) | (geojson_df[filter_surface] >= surface_ha_min))
        & ((surface_ha_max is None) | (geojson_df[filter_surface] <= surface_ha_max))
    ]

    # Sort the filtered DataFrame
    filtered_df.sort_values(by=filter_surface, ascending=False, inplace=True)

    # Calculate the required statistics
    total_features = len(filtered_df)
    features_lt_1 = len(filtered_df[filtered_df[filter_surface] < 1])
    features_1_to_3 = len(
        filtered_df[
            (filtered_df[filter_surface] >= 1) & (filtered_df[filter_surface] < 3)
        ]
    )
    
    features_3_to_8 = len(
        filtered_df[
            (filtered_df[filter_surface] >= 3) & (filtered_df[filter_surface] < 8)
        ]
    )
    features_gt_8 = len(filtered_df[filtered_df[filter_surface] >= 8])

    # mean and median
    mean_surface = filtered_df[filter_surface].mean()
    median_surface = filtered_df[filter_surface].median()

    # Convert the filtered DataFrame to JSON
    filtered_features = filtered_df.head(30).to_json()

    # Create the response dictionary
    response = {
        "total_features": total_features,
        "features_lt_1": features_lt_1,
        "features_1_to_3": features_1_to_3,
        "features_3_to_8": features_3_to_8,
        "features_gt_8": features_gt_8,
        "filtered_features": json.loads(filtered_features),
        "all_features": len(geojson_df),
        "data_file_name": filename,
        "mean_surface": mean_surface,
        "median_surface": median_surface,
    }
    return response


def prepare_data_for_export():
    json_filename = f"{os.path.splitext(filename)[0]}.json"
    with open(json_filename) as f:
        data = json.load(f)
        result = []
        for _, code in data.items():
            response = filter_geojson(code)
            row = {
                "code_cultu": code,
                "total_features": response["total_features"],
                "features_lt_1": response["features_lt_1"],
                "features_1_to_3": response["features_1_to_3"],
                "features_3_to_8": response["features_3_to_8"],
                "features_gt_8": response["features_gt_8"],
                "mean_surface": response["mean_surface"],
                "median_surface": response["median_surface"],
                "data_source": filename,
                "type": is_bio,
            }
            result.append(row)

        return result


In [9]:
category = get_data_category()

filname data/RPG_2-2__GPKG_LAMB93_FXX_2023-01-01/PARCELLES_GRAPHIQUES_4326.json


In [10]:
 # export the data to a JSON file
export_data = prepare_data_for_export()

/tmp/ipykernel_386942/761120573.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by=filter_surface, ascending=False, inplace=True)
/tmp/ipykernel_386942/761120573.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by=filter_surface, ascending=False, inplace=True)
/tmp/ipykernel_386942/761120573.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by=filter_su

In [2]:
export_data